In [2]:
import torch
import numpy as np
import cv2
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision.models import resnet18
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # For showing progress bar
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Dataset(object):
    
    
    def __getitem__(self, index):
        raise NotImplementedError
        
    def __len__(self):
        raise NotImplementedError
        
    def __add__(self, other):
        return ConcatDataset([self,other])

# Define MRI Dataset
class MRI_Dataset(Dataset):
    def __init__(self, tumor_dir, healthy_dir, image_size=(224, 224)):
        self.images = []
        self.labels = []
        self.image_size = image_size

        # Load tumor images
        for filename in os.listdir(tumor_dir):
            img = cv2.imread(os.path.join(tumor_dir, filename))
            if img is not None:
                img = cv2.resize(img, self.image_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                self.images.append(img)
                self.labels.append(1)  # Tumor = 1

        # Load healthy images
        for filename in os.listdir(healthy_dir):
            img = cv2.imread(os.path.join(healthy_dir, filename))
            if img is not None:
                img = cv2.resize(img, self.image_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                self.images.append(img)
                self.labels.append(0)  # Healthy = 0

        self.images = np.array(self.images)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0  # Normalize
        label = self.labels[idx]
        return {'image': torch.tensor(image).permute(2, 0, 1), 'label': torch.tensor(label, dtype=torch.float32)}

# Load dataset paths
train_tumor_dir = './Dataset/Training/aug_tumor'
train_healthy_dir = './Dataset/Training/aug_notumor'
test_tumor_dir = './Dataset/Testing/aug_tumor'  # Separate testing directory
test_healthy_dir = './Dataset/Testing/aug_notumor'  # Separate testing directory

# Create Dataset and DataLoader for training
train_dataset = MRI_Dataset(train_tumor_dir, train_healthy_dir)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create Dataset and DataLoader for testing
test_dataset = MRI_Dataset(test_tumor_dir, test_healthy_dir)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained ResNet18
model = resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),  # Binary classification
    nn.Sigmoid()  # Sigmoid for output probabilities
)

# Initialize model
model = model.to(device)

# Define Loss function and Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set number of epochs (adjust as needed)
epochs = 30

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()  # Zero the gradients before backprop
        
        # Forward pass
        outputs = model(images).squeeze()  # Forward pass through the network
        
        # Compute the loss
        loss = criterion(outputs, labels)  # Compute loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Track the running loss
        running_loss += loss.item()
        
        # Calculate accuracy
        predicted = (outputs > 0.5).float()  # Predicted class: 1 if output > 0.5, else 0
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    # Average loss and accuracy for the epoch
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Validation loop (using the separate test dataset)
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation during validation
        for batch in test_loader:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(images).squeeze()  # Forward pass through the model
            loss = criterion(outputs, labels)  # Compute validation loss
            val_loss += loss.item()
            
            # Calculate validation accuracy
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Average validation loss and accuracy
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

    # Save model after each epoch
    torch.save(model.state_dict(), f'Resnet18False_model_epoch_{epoch+1}.pth')
    print(f"Model saved: model_epoch_{epoch+1}.pth")

# # Save model after all epochs
# torch.save(model.state_dict(), 'final_model.pth')



C:\Users\Omkar Choughule\anaconda3\envs\Py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Omkar Choughule\anaconda3\envs\Py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/30: 100%|████████████████████████████████████████████████████████████████████| 399/399 [00:53<00:00,  7.41it/s]


Train Loss: 0.1970, Accuracy: 92.64%
Validation Loss: 0.2206, Accuracy: 92.13%
Model saved: model_epoch_1.pth


Epoch 2/30: 100%|████████████████████████████████████████████████████████████████████| 399/399 [00:50<00:00,  7.91it/s]


Train Loss: 0.1078, Accuracy: 96.27%
Validation Loss: 0.1601, Accuracy: 94.54%
Model saved: model_epoch_2.pth


Epoch 3/30: 100%|████████████████████████████████████████████████████████████████████| 399/399 [00:50<00:00,  7.91it/s]


...

Train Loss: 0.0215, Accuracy: 99.40%
Validation Loss: 0.0581, Accuracy: 98.61%
Model saved: model_epoch_28.pth


Epoch 29/30: 100%|███████████████████████████████████████████████████████████████████| 399/399 [00:55<00:00,  7.13it/s]


Train Loss: 0.0084, Accuracy: 99.74%
Validation Loss: 0.0515, Accuracy: 98.92%
Model saved: model_epoch_29.pth


Epoch 30/30: 100%|███████████████████████████████████████████████████████████████████| 399/399 [00:56<00:00,  7.12it/s]


Train Loss: 0.0039, Accuracy: 99.86%
Validation Loss: 0.0791, Accuracy: 98.40%
Model saved: model_epoch_30.pth
